# I. Select data inside ellipse

> Francisco Carrasco Varela - Pontificia Universidad Católica de Chile (PUC) - ffcarrasco@uc.cl ⭐

<mark>The following Jupyter Notebook is used to extract and work with Gaia EDR3 / Gaia DR3 data (and other data releases) </mark>

Please read carefully the text above each cell before executing it.

In this part of the project we want the following:
- Use the data available to create a Vector Point Diagram (VPD). Basically a proper motion in RA vs. proper motion in DEC plot.

- Using data from [Harris Catalogue (2010)](https://physics.mcmaster.ca/~harris/mwgc.dat), [Baumgardt  et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.482.5138B/abstract) and [Vasiliev (2019)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.484.2832V/abstract) locate the center of the expected object in the VPD (if available). If the object is not available into the data given by the authors previously mentioned, the user will have to add the coordinates manually.

- Varying the angle inclination (with respect to x-axis) and the size of the ellipse (in semi-minor and major axis) we will try to maximize the number of stars inside of the ellipse.

- Once the parameters that maximize the number of stars inside the ellipse are found, we use those values to create the 'optimal' ellipse.

- We save the data (stars) that lies into the ellipse to use them into the next step of this Notebook.

## 1-. Import data from previous studies

Here we will simply use data saved in files from previous studies such as [Harris Catalogue (2010)](https://physics.mcmaster.ca/~harris/mwgc.dat), [Baumgardt  et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.482.5138B/abstract) and [Vasiliev (2019)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.484.2832V/abstract) based on the name of our object.

This will be useful since in the VPD we will be able to locate the mean proper motion coordinates to locate the center of the ellipse.

If for some reason the object is not found, into the next cell (step/section 2 in this script) mean proper motion coordinates will have to be added manually.

In [1]:
# Import all the libraries we will need

%matplotlib inline
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.offsetbox import AnchoredText
import matplotlib
import numpy as np
import os
from astropy.io import ascii
from astropy.table import vstack, Table
import sys

In [2]:
### This cell gets the properties for OBJ_NAME if it is listed in Vasiliev (2019) file

###################################################
object_name = "NGC104"  # <--- EDITABLE
vasiliev_file = "Vasiliev_2019_Gaia_parameters.dat"
###################################################

class Error(Exception):
    """Base class for other exceptions"""
    pass


class ClusterNotFound(Error):
    """
    Custom error class to indicate the object provided by the user is not in the
    file provided by Vasiliev (2019) and Harris (2010)
    """
    pass


@dataclass(frozen=True, order=True, kw_only=True)
class VasilievData:
    """
    An object/struct used to store data loaded from Vasiliev (2019) file.
    """
    name: str #object name
    RA: float #object RA coordinates (J2000)
    DEC: float #object DEC coordinates (J2000)
    distance: float # Harris (2010) distance in kpc
    losv: float # line of sight velocity in km/s
    err_losv: float # error of line of sight velocity in km/s
    pm_RA : float # mean proper motion RA in mas / yr
    pm_DEC: float # mean proper motion DEC in mas / yr
    err_pm_RA : float # error mean proper motion RA in mas / yr
    err_pm_DEC : float # error mean proper motion DEC in mas / yr
    correlation: float # correlation coefficient (normalized non-diagonal element in the error covariance matrix)
    rh: float # angular scale radius (arcmin)
    N_stars: int # numbers of stars

        
@dataclass
class VasilievList:
    """
    Struct containing a list of VasilievData objects
    """
    data: list[VasilievData] = field(default_factory=list)
        

def get_GC_params(fname: str) -> VasilievList:
    """
    Reads data from file containing Vasiliev (2019) data. And returns a list with their characteristics
    in a dataclass.
    """
    data_type = np.dtype([("name", np.unicode_, 16), ("ra", float), ("dec", float), ("dist",float), ("hrv",float),
                         ("err_hrv", float), ("pmRA", float), ("pmDEC", float), ("err_pmRA", float), ("err_pmDEC", float),
                         ("corr", float), ("r_h",float), ("N_stars", int)])
    gc_names, gc_ra, gc_dec, dist, hrv, e_hrv, pmRA, pmDEC, e_pmRA, e_pmDEC, corr, gc_rh, n_stars = np.loadtxt(fname, 
                                                                                                               dtype=data_type, 
                                                                                                               comments='#',
                                                                                                               unpack=True)    
    data_vasiliev = VasilievList()
    
    for name_it, ra, dec, d, hr, ehr, pmra, pmdec, epmra, epmdec, c, rh_it, n in zip(gc_names, gc_ra, gc_dec, dist, hrv, e_hrv, pmRA, pmDEC, e_pmRA, e_pmDEC, corr, gc_rh, n_stars):
        data = VasilievData(name=name_it, RA=ra, DEC=dec, distance=d, losv=hr, err_losv=ehr, pm_RA=pmra, pm_DEC=pmdec,
                           err_pm_RA=epmra, err_pm_DEC=epmdec, correlation=c, rh=rh_it, N_stars=n)
        data_vasiliev.data += [data]
    
    return data_vasiliev

def get_selected_GC(obj_name: str, data_listed: VasilievList) -> (VasilievData, bool):
    """
    Returns a VasilievData type variable which name matches with that variable.name inside
    Vasiliev Data list.
    If the object (obj_name) is not found in data list (data_listed) then it will return an
    empty variable and it will raise an exception.
    """
    try:
        for item in data_listed.data:
            if item.name.upper() == object_name.upper():
                return item, True
        raise ClusterNotFound
        
    except ClusterNotFound:
        print(f"{obj_name!r} object not found in Globular Cluster list!")
        print("You will have to add coordinates manually in the next cell.")
        print()
        return VasilievData(name="", RA=0., DEC=0., distance=0., losv=0., err_losv=0.,
                            pm_RA=0., pm_DEC=0., err_pm_RA=0, err_pm_DEC=0., correlation=0,
                            rh=0., N_stars=0), False
           
data_list = get_GC_params(vasiliev_file)
obj, success = get_selected_GC(object_name, data_list)

if success:
    n_times = 60
    print("Object detected succesfully!")
    print("-"*n_times)
    print(f"Object name: {obj.name}")
    print(f"Mean Proper Motion RA (mas/yr): {obj.pm_RA} +- {obj.err_pm_RA}")
    print(f"Mean Proper Motion DEC (mas/yr): {obj.pm_DEC} +- {obj.err_pm_DEC}")
    print("-"*n_times)

Object detected succesfully!
------------------------------------------------------------
Object name: NGC104
Mean Proper Motion RA (mas/yr): 5.237 +- 0.039
Mean Proper Motion DEC (mas/yr): -2.524 +- 0.039
------------------------------------------------------------


## 2-. Read filtered file from previous step

Now, continuing from the previous step, we will read the previously saved file (where we had a 'Raw' file and a 'Filtered' file). 
Since we are using Astropy it is recommended to use te 'ascii.escv' format since it allows Astropy to read units (if available) in a faster and easier way.

In this case we will open the file located at *./OBJ_NAME/OBJ_NAME_f_data.dat* since it has values/lower errors for our interest that we have previously filtered.

In [3]:
def check_if_file_exists(filename_path: str) -> None:
    """
    Checks if a file with filtered data that should have been created in the previous step
    of this Notebook is created. If it is not created it will exit the program.
    """
    isExist = os.path.exists(filename_path)
    if not isExist:
        print("You must fully run the previous step in this Notebook and create a file with ", end='')
        print("filtered data before running this cell.")
        sys.exit("Create filtered file in previous step and retry.")
    return


filename_filtered_path = f"./Objects/{object_name.upper()}/{object_name.upper()}_f_data.dat"

# Check if the filtered file created in the previous step of thi Notebook exists
check_if_file_exists(filename_filtered_path)

gaia_data = Table.read(filename_filtered_path, format='ascii.ecsv') # get data from previous Notebook step
print('Data read sucessfully')

Data read sucessfully


Once the data has been read we will apply the following idea to the ellipse:

- Draw an elipse in the VPD.
- If the point (star) is inside the ellipse, add +1 to a counter that counts stars inside ellipse.
- If the point is outside the ellipse, add +1 to a counter that counts stars outside ellipse.
- Get the parameters that maximize the number of stars inside the ellipse.
- Once those parameters are found, use them to create the 'optimized' ellipse and save data inside it for the next step.

In [ ]:
# In this cell we will simply define some classes that we will use in the next
# cell. Since the next cell takes its time to run, it is better to define them
# in this place

@dataclass(order=True, kw_only=True)
class EllipseClass():
    """
    A class containing all parameters of an ellipse for our interest
    """
    center_x: float # x-axis center coordinate
    center_y: float # y-axis center coordinate
    width: float # in mas / yr
    height: float # in mas / yr
    inclination: float # angle inclination in degrees


@dataclass(frozen=True, order=True, kw_only=True)
class IteratorClass():
    """
    A class used to change iterators cycles/parameters
    """
    minimum: float
    maximum: float
    n_step: int

You may want to edit some values in the next cell... (look for *# <--- EDITABLE* in code line)

In [ ]:
# This cell is a Montecarlo selection, go for a coffee.
# It will take some time...

def checkIfMax(value: int, current_max: int) -> (int, bool):
    """
    Simple function that checks if a value is bigger than a current one.
    If the value is greater than the current maximum, it returns the value.
    Otherwise it returns the maximum.
    """
    if value > current_max:
        return value, True
    return current_max, False


def DefineEllipse(x: np.ndarray, y: np.ndarray, x_center: float, y_center: float, 
                  ell_width: float, ell_height: float, angle_inclination: float, 
                  ) -> np.ndarray:
    """
    Create an ellipse with center given with coordinates (x_center, y_center).
    Also, a width and height (x and y-axis, respectively) must be passed, given
    an 'incilnation' angle.
    """
    
    cos_angle = np.cos(np.radians(180.-angle_inclination))
    sin_angle = np.sin(np.radians(180.-angle_inclination))

    xc = x - x_center
    yc = y - y_center

    xct = xc * cos_angle - yc * sin_angle
    yct = xc * sin_angle + yc * cos_angle 

    rad_cc = (xct**2/(ell_width/2.)**2) + (yct**2/(ell_height/2.)**2)
    return rad_cc


def countStarsInsideEllipse(obj: VasilievData, gaiaData: Table) -> EllipseClass:
    """
    A simple function that count stars inside an ellipse. It will return an Ellipse
    object with parameters that maximizes the numbers of stars/objects within an
    ellipse in a Vector Point Diagram; where the ellipse center will be given by
    parameters given in Vasiliev (2019) file (if available).
    """
    inside_stars, outside_stars = 0, 0
    
    # Create a copy of the arrays containing positions (x,y) in VPD
    x, y = np.asarray(gaiaData['pmra']), np.asarray(gaiaData['pmdec'])

    ### <-------------------------->
    ### Change the values in the cycle for those you think are convenient
    ### Parameters for ellipse width
    width_min, width_max, width_nsteps = 1., 1.8, 15 # in mas / yr <---------EDITABLE
    width_it = IteratorClass(minimum=width_min, maximum=width_max, n_step=width_nsteps)

    ### Parameters for ellipse height
    height_min, height_max, height_nsteps = 1.5, 2., 20 # in mas / yr <---------EDITABLE
    height_it = IteratorClass(minimum=height_min, maximum=height_max, n_step=height_nsteps)
    
    ### Parameters for ellipse inclination
    incl_min, incl_max, incl_nsteps = -89., 89., 200 # in degrees <---------EDITABLE
    angle_incl_it = IteratorClass(minimum=incl_min, maximum=incl_max, n_step=incl_nsteps)

    ### <-------------------------->
    
    width_range = np.linspace(width_it.minimum, width_it.maximum, width_it.n_step)
    height_range = np.linspace(height_it.minimum, height_it.maximum, height_it.n_step)
    angle_range = np.linspace(angle_incl_it.minimum, angle_incl_it.maximum, angle_incl_it.n_step)

    max_in_stars = 0
    
    for w_it in width_range:
        for h_it in height_range:
            for angle_it in angle_range:
                if w_it == h_it:
                    continue # due to tidal forces, object in VPD will have an elliptic form
                             # also, applying an inclination to a circle is not very useful...
                counter_in, counter_out = 0, 0
                ellipse_zone = DefineEllipse(x, y, obj.pm_RA, obj.pm_DEC, w_it, h_it, angle_it)
                for r in ellipse_zone:
                    if r <= 1:
                        counter_in += 1
                    if r > 1:
                        counter_out += 1
                max_in_stars, save_parameters = checkIfMax(counter_in, max_in_stars)
                if save_parameters:
                    ellipse_parameters = EllipseClass(center_x=obj.pm_RA, center_y=obj.pm_DEC, width=w_it,
                                                     height=h_it, inclination=angle_it)
    
    return ellipse_parameters
 

    
ellipse = countStarsInsideEllipse(obj, gaia_data)
print("\"Optimized\" ellipse parameters:")
print(f"-> Center coordinates in VPD: ({ellipse.center_x:.2f}, {ellipse.center_y:.2f})")
print(f"-> Width: {ellipse.width:.2f} mas / yr\n-> Height: {ellipse.height:.2f} mas / yr")
print(f"-> Angle inclination: {ellipse.inclination:.2f} deg")
